## 토픽 모델링
* 실습을 위해 pyLDAvis 설치가 필요합니다. 
* colab사용시 설치 후에도 제대로 동작하지 않거나 오류가 나면 런타임을 재실행 해주세요.

In [ ]:
# !pip install -U -q pyLDAvis

In [ ]:
# ignore warnings
import warnings
warnings.filterwarnings("ignore")

## 라이브러리 로드

In [ ]:
# 필요 라이브러리를 로드합니다.
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 로드

In [ ]:
# 수집한 데이터셋을 불러옵니다.
df = pd.read_csv("https://bit.ly/seoul-120-text-csv")
df.shape

In [ ]:
# 결측치가 있다면 제거합니다.
df = df.dropna()
df.shape

In [ ]:
# 결측치를 확인합니다.
df.isnull().sum()

## 문서 만들기
* 제목과 내용을 함께 사용합니다.

In [ ]:
df["문서"] = df["제목"] + " " + df["내용"]

## 벡터화

* [Bag-of-words model - Wikipedia](https://en.wikipedia.org/wiki/Bag-of-words_model)


## CountVectorizer

* analyzer : 단어, 문자 단위의 벡터화 방법 정의
* ngram_range : BOW 단위 수 (1, 3) 이라면 1개~3개까지 토큰을 묶어서 벡터화
* max_df : 어휘를 작성할 때 문서 빈도가 주어진 임계값보다 높은 용어(말뭉치 관련 불용어)는 제외 (기본값=1.0)
    * max_df = 0.90 : 문서의 90% 이상에 나타나는 단어 제외
    * max_df = 10 : 10개 이상의 문서에 나타나는 단어 제외
* min_df : 어휘를 작성할 때 문서 빈도가 주어진 임계값보다 낮은 용어는 제외합니다. 컷오프라고도 합니다.(기본값=1.0)
    * min_df = 0.01 : 문서의 1% 미만으로 나타나는 단어 제외
    * min_df = 10 : 문서에 10개 미만으로 나타나는 단어 제외
* stop_words : 불용어 정의
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [ ]:
# 단어들의 카운트(출현 빈도(frequency))로 여러 문서들을 벡터화하기 위해 CountVectorizer를 불러옵니다.
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(stop_words=["돋움", "경우", "또는"])

### 참고: fit, transform, fit_transfrom의 차이점
- fit(): 원시 문서에 있는 모든 토큰의 어휘 사전을 배운다
- transform(): 문서를 문서 용어 매트릭스로 변환, transform 이후엔 매트릭스로 변환되어 숫자형태로 변경
- fit_transform(): 어휘 사전을 배우고 문서 용어 매트릭스를 반환, fit 다음에 변환이 오는 것과 동일하지만 더 효율적으로 구현

* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.fit_transform

In [ ]:
# fit_transform을 사용하여 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성합니다.
dtm_cv = cv.fit_transform(df["문서"])

In [ ]:
# cv.vocabulary_ 를 봅니다.
# cv.vocabulary_

In [ ]:
cv_cols = cv.get_feature_names()

In [ ]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 one-hot-vector를 확인합니다.
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인합니다.

pd.DataFrame(dtm_cv.toarray(), columns=cv_cols).sum().sort_values()

## BOW<sup>bag of word</sup> 잠재 디리클레 할당(Latent Dirichlet Allocation, LDA)

* API documentation: https://pyldavis.readthedocs.io/en/latest/modules/API.html

In [ ]:
# 정답인 '분류'의 유일한 값을 확인하여 주제 수를 확인합니다.
df["분류"].value_counts()

In [ ]:
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 확인하는 잠재 디리클레 분석(LDA)을 불러옵니다.
# n_components에 넣을 하이퍼파라미터 NUM_TOPICS로 주제수를 설정합니다.(기본값=10)
# max_iter는 훈련 데이터(epoch라고도 함)에 대한 최대 패스 수입니다.(기본값=10)

from sklearn.decomposition import LatentDirichletAllocation

NUM_TOPICS = 10
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, random_state=42)

In [ ]:
# LDA_model 에 dtm_cv 를 넣어 학습합니다.
LDA_model.fit(dtm_cv)

### pyLDAvis

In [ ]:
# !pip install -U -q pyLDAvis

In [ ]:
# 토픽 모델링에 이용되는 LDA 모델의 학습 결과를 시각화하는 Python 라이브러리인 pyLDAvis를 불러옵니다.
# mds(Multi-Dimensional Scaling)는 데이터 포인트 간의 거리를 보존하면서 차원을 축소하는 기법입니다.
# t-SNE(t-Stochastic Neighbor Embedding)은 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용합니다.

import pyLDAvis.sklearn

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model, dtm_cv, cv, mds='tsne')

## TF-IDF(Term Frequency - Inverse Document Frequency)

## TfidfVectorizer

TF-IDF 인코딩은 단어를 갯수 그대로 카운트하지 않고 모든 문서에 공통적으로 들어있는 단어(낮은 구별력)의 경우 가중치를 축소하는 방법

매개변수
* norm='l2' 각 문서의 피처 벡터를 어떻게 벡터 정규화 할지 정한다. 
    - L2 : 벡터의 각 원소의 제곱의 합이 1이 되도록 만드는 것이고 기본 값
    - L1 : 벡터의 각 원소의 절댓값의 합이 1이 되도록 크기를 조절
* smooth_idf=False
    - 피처를 만들 때 0으로 나오는 항목에 대해 작은 값을 더해서(스무딩을 해서) 피처를 만들지 아니면 그냥 생성할지를 결정
* sublinear_tf=False
* use_idf=True
    - TF-IDF를 사용해 피처를 만들 것인지 아니면 단어 빈도 자체를 사용할 것인지 여부
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer


In [ ]:
# TF-IDF 방식으로 단어의 가중치를 조정한 BOW 인코딩하여 벡터화하기 위해 TfidfVectorizer를 불러옵니다.

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=["돋움", "경우", "또는", "있습니다", "있는", "합니다"])
tfidf

In [ ]:
# 문장에서 노출되는 feature(특징이 될만한 단어) 수를 합한 변수 Document Term Matrix(이하 dtm)를 생성합니다.
dtm_tfidf = tfidf.fit_transform(df["문서"])

In [ ]:
# tfidf.vocabulary_ 의 번호는 정렬 순으로 되어 있습니다.
# tfidf.vocabulary_
cols_tfidf = tfidf.get_feature_names()

In [ ]:
# dtm_tf를 axis=0(수직 방향으로) 기준으로 합계를 낸 dist 변수를 생성합니다.
# dist 변수를 vocabulary_ 순으로 정렬하여 비율을 확인합니다.
dist = np.sum(dtm_tfidf, axis=0)
pd.DataFrame(dist, columns=cols_tfidf).T.sort_values(by=0).tail(10)

In [ ]:
# 각 row에서 전체 단어가방에 있는 어휘에서 등장하는 단어에 대한 가중치를 적용한 vector를 확인합니다.
# toarray()로 희소 행렬(sparse matrix, 행렬의 값이 대부분 '0'인 행렬)을 NumPy array 배열로 변환하여 값을 확인합니다.
pd.DataFrame(dtm_tfidf.toarray(), columns=cols_tfidf)

## TF-IDF 잠재 디리클레 할당(LDA)

In [ ]:
# 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지를 확인하는 잠재 디리클레 분석(LDA)을 불러옵니다.
# n_components에 넣을 하이퍼파라미터 NUM_TOPICS로 주제수를 설정합니다.(기본값=10)
# max_iter는 훈련 데이터(epoch라고도 함)에 대한 최대 패스 수입니다.(기본값=10)

NUM_TOPICS = 10 
LDA_model = LatentDirichletAllocation(n_components=NUM_TOPICS, 
                                      max_iter=30, 
                                      random_state=42)

In [ ]:
# dtm_tfidf 를 LDA_model로 학습시킵니다.
LDA_model.fit(dtm_tfidf)

In [ ]:
# 토픽 모델링에 이용되는 LDA 모델의 학습 결과를 시각화하는 Python 라이브러리인 pyLDAvis를 불러옵니다.
# mds(Multi-Dimensional Scaling)는 데이터 포인트 간의 거리를 보존하면서 차원을 축소하는 기법입니다.
# t-SNE(t-Stochastic Neighbor Embedding)은 고차원 데이터를 특히 2, 3차원 등으로 줄여 가시화하는데에 유용합니다.

pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(LDA_model, dtm=dtm_tfidf, vectorizer=tfidf, mds='tsne')

## 코사인 유사도
* API Document: https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html

In [ ]:
# df.head()

In [ ]:
# 등장 빈도에 기반하여, 코사인 유사도 알고리즘 적용해봅니다.
# 첫 행의 "아빠 육아 휴직 장려금"과 비슷한 데이터 정렬해봅니다.

from sklearn.metrics.pairwise import cosine_similarity

similarity_simple_pair = cosine_similarity(dtm_tfidf[0] , dtm_tfidf)
result_list = similarity_simple_pair.tolist()[0]

In [ ]:
# result_list를 "유사도" 파생변수로 생성하고 유사도가 높은 순으로 정렬합니다.

df["유사도"] = result_list
df[["분류", "제목", "유사도"]].sort_values(by="유사도", ascending=False).head(10)